In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Read data

In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

# Generate X Y 

In [3]:
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(604, 1704)
(604, 5112)


In [4]:
x1Restruct_Fun=x1
x2Restruct_Fun=x2
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(604, 1704)
(604, 5112)


In [5]:
YRestruct_Fun=np.zeros([601,5112])
for j in range (0,71):
    for i in range(0,601):
        YRestruct_Fun[i][72*j:72*j+24]=x1Restruct_Fun[1+i][24*j:24*j+24]
        YRestruct_Fun[i][72*j+24:72*j+48]=x1Restruct_Fun[1+i+1][24*j:24*j+24]
        YRestruct_Fun[i][72*j+48:72*j+72]=x1Restruct_Fun[1+i+2][24*j:24*j+24]
print(np.shape(YRestruct_Fun))

(601, 5112)


In [6]:
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(601, 6816)


In [7]:
Xhat=np.zeros([540,6816])
Yhat=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        Xhat[i][j]=XRestruct_Fun[i][j]
        Yhat[i][j]=YRestruct_Fun[i][j]
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(540, 6816)
Yhat
(540, 5112)


# PCA & Linear regression

In [8]:
per = np.zeros(99)
for i in range (0,99):
    per[i]=0.01*(i+1)
per_error = np.zeros(99)

In [9]:
for z in range (0,99):
    pca = PCA(n_components=per[z])
    pca.fit(Xhat)
    x_pca=pca.transform(Xhat)

    model = LinearRegression()
    model.fit(x_pca, Yhat)

    """
    generate new cal
    """
    x1=np.zeros([604,1704])
    x2=np.zeros([604,5112])
    #x1 (0th~1703th column as x)
    for i in range (0,604):
        for j in range (0,71):
            a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
            for k in range (0,24):
                x1[i][j*24+k]=a[k]

    #x2 (1704th~6816th column as x)
    for i in range (1,604):
        b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        for j in range(0,5112):
            x2[i-1][j]=b[j]
    x1Restruct_Fun=x1
    x2Restruct_Fun=x2
    
    XRestruct_Fun=np.zeros([601,6816])
    for i in range (0,601):
        for j in range (0,1704):
            XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
        for j in range (1704,6816):
            XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]

    xf = np.zeros([61,6816])
    for i in range (0,61):
        for j in range(0,6816):
            xf[i][j] = XRestruct_Fun[i+540][j]

    new_xcal=pca.transform(xf)
    new_xfit=model.predict(new_xcal)

    #
    """
    old data
    """
    old_x=np.zeros([601,5112])
    old_y=np.zeros([601,5112])
    #data_cal
    for i in range (1,602):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i-1]=a
    #data_obs
    for i in range (1,602):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i-1]=a
        
    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+540][j]
            old_y2[i][j]=old_y[i+540][j]
            fit_fx2[i][j]=new_xfit[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)    

    per_error[z] = after_error2

# Result

In [10]:
print(per_error)

[6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408
 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408
 6.38954351 6.38954351 6.38954351 6.38954351 6.38954351 6.38954351
 6.38954351 6.22348962 6.22348962 6.22348962 6.22348962 6.08456267
 6.08456267 6.07643685 6.07643685 6.07643685 5.89402931 5.68002756
 5.68002756 5.62004936 5.63702169 5.62717568 5.62302511 5.62302511
 5.57182115 5.52282439 5.45632312 5.43828866 5.43359993 5.44846571
 5.43481479 5.31026027 5.25796781 5.22436342 5.22722523 5.3165738
 5.36139257 5.3549918  5.33443709 5.37461059 5.33616535 5.34546282
 5

In [11]:
print('on', np.argmin(per_error)+1, '% PCA', 'have the min MAE error :', min(per_error))

on 82 % PCA have the min MAE error : 5.224363416035845
